In [15]:
#load data
import pandas as pd

data = pd.read_csv('../dataset/raw/DP15_Bvot_T1T2.csv', sep=';')

df = data.copy()
original_data_canton_FR = pd.read_excel(r'../dataset/raw/Dep_15_Resultats_T1_complet.xlsx', sheet_name='Cantons', header=2)
# original_data_Bvot_FR        = pd.read_csv('../dataset/inputs/XDataFR_Bvot.csv', sep=';')
# original_data_Bvot_targets_FR = pd.read_csv('../dataset/labels/yDataFR_Bvot.csv', sep=';')

data_canton_FR = original_data_canton_FR.copy()
# data_Bvot_FR   = original_data_Bvot_FR.copy()
# data_Bvot_targets_FR = original_data_Bvot_targets_FR.copy()

/home/jaman/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
def saveData(data, loc):
    # save cher_data as excel
    writer = pd.ExcelWriter(loc)
    
    # write dataframe to excel
    data.to_excel(writer)

    # save the excel
    writer.save()

In [4]:
df.columns

Index(['NUMTOUR', 'CODDPT', 'CODSUBCOM', 'LIBSUBCOM', 'CODBURVOT', 'CODCAN',
       'LIBCAN', 'NBRINS', 'NBRVOT', 'NBREXP', 'NUMDEPCAND', 'LIBLISEXT',
       'CODNUA', 'NBRVOIX'],
      dtype='object')

In [5]:
df_t1 = df[df.NUMTOUR == 1]
df_t2 = df[df.NUMTOUR == 2]

In [6]:
df_t1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289173 entries, 0 to 289172
Data columns (total 14 columns):
NUMTOUR       289173 non-null int64
CODDPT        289173 non-null object
CODSUBCOM     289173 non-null int64
LIBSUBCOM     289173 non-null object
CODBURVOT     289173 non-null object
CODCAN        289173 non-null int64
LIBCAN        289173 non-null object
NBRINS        289173 non-null int64
NBRVOT        289173 non-null int64
NBREXP        289173 non-null int64
NUMDEPCAND    289173 non-null int64
LIBLISEXT     289173 non-null object
CODNUA        289173 non-null object
NBRVOIX       289173 non-null int64
dtypes: int64(8), object(6)
memory usage: 43.1+ MB


In [7]:
df_t2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131891 entries, 289173 to 421063
Data columns (total 14 columns):
NUMTOUR       131891 non-null int64
CODDPT        131891 non-null object
CODSUBCOM     131891 non-null int64
LIBSUBCOM     131891 non-null object
CODBURVOT     131891 non-null object
CODCAN        131891 non-null int64
LIBCAN        131891 non-null object
NBRINS        131891 non-null int64
NBRVOT        131891 non-null int64
NBREXP        131891 non-null int64
NUMDEPCAND    131891 non-null int64
LIBLISEXT     131891 non-null object
CODNUA        131891 non-null object
NBRVOIX       131891 non-null int64
dtypes: int64(8), object(6)
memory usage: 20.1+ MB


In [12]:
saveData(df_t1[df_t1.CODDPT == str(2)], '../dataset/temp/dep2_t1.xlsx')
saveData(df_t2[df_t2.CODDPT == str(2)], '../dataset/temp/dep2_t2.xlsx')

In [124]:
data_Bvot_targets_FR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60975 entries, 0 to 60974
Data columns (total 22 columns):
CODDPT       60975 non-null object
CODCAN       60975 non-null int64
CODSUBCOM    60975 non-null int64
CODBURVOT    60975 non-null object
BC-COM       60974 non-null float64
BC-DIV       60974 non-null float64
BC-DLF       60974 non-null float64
BC-DVD       60974 non-null float64
BC-DVG       60974 non-null float64
BC-EXD       60974 non-null float64
BC-FG        60974 non-null float64
BC-FN        60974 non-null float64
BC-MDM       60974 non-null float64
BC-PG        60974 non-null float64
BC-RDG       60974 non-null float64
BC-SOC       60974 non-null float64
BC-UC        60974 non-null float64
BC-UD        60974 non-null float64
BC-UDI       60974 non-null float64
BC-UG        60974 non-null float64
BC-UMP       60974 non-null float64
BC-VEC       60974 non-null float64
dtypes: float64(18), int64(2), object(2)
memory usage: 10.2+ MB


In [37]:
def getNbBinomes(data):
    return len([header for header in data.columns if "Binôme" in header])

def explodeLines(data):
    initdf = data[['Code du département', 'Libellé du département', 'Code du canton', 
            'Libellé du canton', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants',
            '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins',
            '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot']]

    headers = ['N°Panneau', 'Nuance', 'Binôme', 'Sièges', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    df = pd.DataFrame()
    for i in range(getNbBinomes(data)):
        partidf = data[[h+'.'+str(i) if i!=0 else h for h in headers]]
        partidf = pd.concat([initdf, partidf], axis=1)
        partidf.columns = pd.Index(initdf.columns.values.tolist() + headers)
        df = pd.concat([df, partidf])
    
    # Remove useless rows   
    df = df.dropna(how='all', subset=headers)
    
    return df

In [38]:
data_canton_FR = explodeLines(data_canton_FR)
# tmp = fr_data[fr_data['Code du département']==63]
# tmp = tmp[tmp['Code du canton']==4]
# tmp

tmp = fr_data.loc[(data_canton_FR['Code du département']==1 ) & (data_canton_FR['Code du canton']==1) ]
tmp

,Code du département,Libellé du département,Code du canton,Libellé du canton,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,N°Panneau,Nuance,Binôme,Sièges,Voix,% Voix/Ins,% Voix/Exp
0,1,AIN,1,Ambérieu-en-Bugey,19027,9588,50.39,9439,49.61,281,1.48,2.98,100,0.53,1.06,9058,47.61,95.96,2.0,BC-COM,Mme BRESSON Christine et M. ROBERT Patrice,,684.0,3.59,7.55
0,1,AIN,1,Ambérieu-en-Bugey,19027,9588,50.39,9439,49.61,281,1.48,2.98,100,0.53,1.06,9058,47.61,95.96,5.0,BC-UG,M. BOUCHON Gilbert et Mme PIDOUX Catherine,,2108.0,11.08,23.27
0,1,AIN,1,Ambérieu-en-Bugey,19027,9588,50.39,9439,49.61,281,1.48,2.98,100,0.53,1.06,9058,47.61,95.96,3.0,BC-VEC,Mme BOUDIB Emmanuelle et M. NAVARRO Jean,,735.0,3.86,8.11
0,1,AIN,1,Ambérieu-en-Bugey,19027,9588,50.39,9439,49.61,281,1.48,2.98,100,0.53,1.06,9058,47.61,95.96,1.0,BC-UD,Mme CASTELLANO Sandrine et M. FORTIN Christophe,,2572.0,13.52,28.39
0,1,AIN,1,Ambérieu-en-Bugey,19027,9588,50.39,9439,49.61,281,1.48,2.98,100,0.53,1.06,9058,47.61,95.96,4.0,BC-FN,Mme PAGNEUX Nadège et M. VERGNAULT Jean-Pierre,,2959.0,15.55,32.67


In [36]:
def getNuanceOfElected(data, col_siege='Sièges', col_nuance='Nuance'):
    elected = data[data[col_siege]=='Elus']
    return list(elected[col_nuance].unique()) if len(elected)!=0 else None

def filterBestNuances(data, col_nuance='Nuance', criteria=12.50):
    bestCandidat = data[data['% Voix/Ins']>= criteria]
    if bestCandidat.empty or len(bestCandidat)==1:
        bestCandidat = data.sort_values(by='Voix', ascending=False).iloc[0:2,:]

    return list(bestCandidat[col_nuance].unique())

def getDuels(data, dep, col_dep='Code du département', col_canton='Code du canton', col_siege='Sièges', col_nuance='Nuance'):
    data = data[data[col_dep]==dep]
    duels = dict()
    for canton in data[col_canton].unique():
        data_canton = data[data[col_canton]==canton]
        # allow to know if there is a majority in the canton
        elected = getNuanceOfElected(data_canton, col_siege=col_siege, col_nuance=col_nuance) 
        if elected is not None:
            duels[str(canton)]= elected
        else:
            duels[str(canton)] = filterBestNuances(data_canton)
    return duels



In [39]:
duels = dict()
for dep in data_canton_FR['Code du département'].unique():
    duels[str(dep)]= getDuels(data_canton_FR, dep)
    if duels[str(dep)]==[]:
        print('empty list for dep : ', dep)

In [41]:
duels['2']

{'1': ['BC-UG', 'BC-FN'],
 '2': ['BC-UG', 'BC-FN'],
 '3': ['BC-UG', 'BC-FN'],
 '4': ['BC-UG', 'BC-UD', 'BC-FN'],
 '5': ['BC-UMP', 'BC-FN'],
 '6': ['BC-UG', 'BC-UD', 'BC-FN'],
 '7': ['BC-SOC', 'BC-UD', 'BC-FN'],
 '8': ['BC-UG', 'BC-FN'],
 '9': ['BC-UG', 'BC-FN'],
 '10': ['BC-UG', 'BC-FN'],
 '11': ['BC-UG', 'BC-UD', 'BC-FN'],
 '12': ['BC-UG', 'BC-UD', 'BC-FN'],
 '13': ['BC-UD', 'BC-FN'],
 '14': ['BC-UD', 'BC-FN'],
 '15': ['BC-UD', 'BC-FN'],
 '16': ['BC-UG', 'BC-UD', 'BC-FN'],
 '17': ['BC-UG', 'BC-UD', 'BC-FN'],
 '18': ['BC-UG', 'BC-FN'],
 '19': ['BC-UDI', 'BC-UMP', 'BC-FN'],
 '20': ['BC-FN'],
 '21': ['BC-FN', 'BC-UMP']}

In [119]:

# retourne un dataset 
def getTrainSets(X, y, duel, col_canton='CODCAN', col_dep='Code du département'):
    mask_dep    = pd.Series([False]*len(X))
    mask_canton = pd.Series([False]*len(X))
    for dep in X[col_dep].unique():
        for canton in X[X[col_dep]==dep][col_canton].unique(): #pour chaque canton du departement
            #on considere que les cantons qui ont comme partis les partis du duel
            if all(nuance in X.loc[(X[col_dep]==dep) & (X[col_canton]==canton)] for nuance in duel):
                mask_dep    |= X[col_dep]==dep
                mask_canton |= X[col_canton]==canton

    # filtre les departement dont aucun canton contient notre duel
    X= X[mask_dep]
    y = y[mask_dep]
    return (X[mask_canton], y[mask_canton])
                        


In [ ]:
#